# Day 7

[Day 7 description](https://adventofcode.com/2018/day/7)

Well, this one was the first challenge, at least in the second part. It's a graph-ordering problem. We can model every sentence `Step x must be finished before step y can begin` implies that there's a DAG with nodes `x` and `y` and with a directed path from `x` to `y`. We need to collect all information and build corresponding graph.

In general, we want to explore the graph. A node is available when all preceeding nodes are completed. Hence, we start with nodes that are immediatly available, then we upgrade the explored nodes list. If two steps are available, choose the first one alphabetically (i.e. I need to re-compute the list of available steps after every node completed).

In [1]:
import re
from string import ascii_uppercase

In [2]:
get_row = re.compile("Step ([A-Z]+) must be finished before step ([A-Z]+) can begin")
def parse_row(s):
    return get_row.findall(s.strip())[0]

In [3]:
with open('AOC2018_07_input.txt') as f:
    edges = [parse_row(r) for r in f.readlines()]

The `next_step` function will process current edges, current steps and produce the next step (while trowing away all consumed edges). When only one edge is missing, we collect both nodes.

In [4]:
def next_step(available_edges, steps_done):
    if len(available_edges) == 1:
        return list(), steps_done + list(available_edges[0])
    ss = {e[0] for e in available_edges}
    ts = {e[1] for e in available_edges}
    last_step = min(ss.difference(ts))
    new_edges = [e for e in available_edges if not e[0] == last_step]
    return new_edges, steps_done + [last_step]

In [5]:
new_edges = list(edges)
new_steps = list()
while len(new_edges) > 0:
    new_edges, new_steps = next_step(new_edges, new_steps)
"".join(new_steps)

'AEMNPOJWISZCDFUKBXQTHVLGRY'

Now for the real challenge! The second part is much tougher. We need to parallelize the job to many workers (5); moreover, every node has it's own timing, i.e., `60 + ordinal number of char`. So for example, step `A` will take `60+1` seconds, `B` will take `60+2`, ..., `Z` will take `60+26`.

Again, a step cannot start if every previous step is not completed, which means that we need to care of the fact that a node have to be visited and a certain amount of time should have passed. When no worker is available or no node is available, we need to wait for a node to be completed (of course, time will pass for other workers too). This will free a worker (for sure) and maybe give access to more nodes.

Instead of using a class for workers, I use a pair `(n, C)` where `n` is the remaining time and `C` is the current task; so after waiting `n` minutes, task `C` is completed and worker is ready for next task.

As a little trick: we add one extra node (`*`) as a target for every "final" node. This will make it easier to write the logic.

In [6]:
def reachable_nodes(edges, current_nodes, workers):
    avoid = set(current_nodes)
    edges_ = [e for e in edges if not e[0] in avoid]
    reachables = {e[0] for e in edges_}.difference({e[1] for e in edges_})
    reachables = reachables.difference({j for _, j in workers})
    return reachables

def assign_to_worker(reachables, workers):
    reachables = sorted(reachables)
    new_workers = list()
    assigned = list()
    for worker in workers:
        if worker[0] != 0 or len(reachables) == 0:
            new_workers.append(worker)
        else:
            next_task, *reachables = reachables
            new_workers.append((node_time[next_task], next_task))
            assigned.append(next_task)
    return new_workers, assigned

def wait(workers):
    wait_time = min([t for t, _ in workers if t > 0])
    new_workers, finished = list(), list()
    for worker in workers:
        wtime, wtask = worker
        if wtime <= wait_time:
            new_workers.append((0, ''))
            if wtime == wait_time: finished.append(wtask)
        else:
            new_workers.append((wtime - wait_time, wtask))
    return new_workers, wait_time, finished

def next_step_workers(edges, steps_done, workers, time):
    reachables = reachable_nodes(edges, steps_done, workers)
    if len(reachables) > 0:
        workers, assigned = assign_to_worker(reachables, workers)
    workers, wait_time, finished = wait(workers)
    edges = [e for e in edges if not e[0] in finished]
    new_steps = steps_done + finished
    return edges, new_steps, workers, time + wait_time

In [7]:
node_time = {v: 60+k+1 for k, v in enumerate(ascii_uppercase)}
n_workers = 5
workers = [(0, '') for _ in range(n_workers)]

steps = list()
total_time = 0
edges_extended = list(edges) + [(x, '*') for x in {e[1] for e in edges}.difference(e[0] for e in edges)]

In [8]:
while len(edges_extended) > 0:
    edges_extended, steps, workers, total_time = next_step_workers(edges_extended, steps, workers, total_time)
    
total_time + max([wtime for wtime, _ in workers])

1081